## Poolish Baguette

- bulk fermentation: 2hrs, fold once
- divide & preshape: 10-30 min
- shape
- final fermentation: 1-1.5 hrs
- bake: ~25min


In [10]:
import pandas as pd
grams_per_ounce = 28.3495

In [11]:
formatter = {
    'grams': lambda x: f"{x:,.0f}" if x > 10 else f"{x:.2f}",
    'oz': lambda x: f"{x:,.0f}" if x > 100 else f"{x:.2f}",
    'baker%': lambda x: f"{x:.0f}" if x > 2 else f"{x:.1f}"
}

In [12]:
num_loaves = 4
num_pounds = 0
num_ounces = 14
num_grams = 0

In [13]:
loaf_weight = (num_pounds * 16 + num_ounces) * grams_per_ounce + num_grams
total_weight = (num_loaves * loaf_weight)
print ("{:,.0f} loaves at {:,.0f} grams".
       format(num_loaves, loaf_weight, num_pounds, num_ounces))
print ("total weight: {:,.2f} grams".format(total_weight,total_weight * grams_per_ounce))

4 loaves at 397 grams
total weight: 1,587.57 grams


In [14]:
pre_fermented_flour = .33
formula = pd.DataFrame.from_dict({
    "flour": 100, 
    "water": 66, 
    "yeast": .363,
    "salt": 2,
    "malt": 0.5
}, orient='index', columns=["baker%"])
formula_total = formula['baker%'].sum()
print("overall formula total = %s%%" % formula_total)

overall formula total = 168.863%


In [15]:
flour_weight = formula.at['flour', 'baker%'] / formula_total * total_weight
formula['grams'] = formula.apply(lambda row: row['baker%'] * flour_weight / 100, axis=1)
formula['oz'] = formula.apply(lambda row: row['grams'] / grams_per_ounce, axis=1)
print ("overall formula")

formula[['baker%','grams','oz']].style.format(formatter)

overall formula


,baker%,grams,oz
flour,100,940,33.16
water,66,621,21.89
yeast,0.4,3.41,0.12
salt,2.0,19,0.66
malt,0.5,4.70,0.17


In [16]:
poolish = pd.DataFrame.from_dict({
    "flour": 100,
    "water": 100, 
    "yeast": 0.2
}, orient='index', columns=["baker%"])
print ("poolish")
poolish_total = poolish['baker%'].sum()
print("poolish total = %s%%" % poolish_total)
poolish.style.format(formatter)

poolish
poolish total = 200.2%


,baker%
flour,100
water,100
yeast,0.2


In [17]:
poolish_flour = total_weight / formula_total * formula.at['flour','baker%'] * pre_fermented_flour
poolish_weight = poolish_flour * poolish_total / poolish.at['flour','baker%']
print ("poolish weight = {:,.0f} grams".format(poolish_weight))
poolish['grams'] = poolish.apply(lambda row: row['baker%']*poolish_weight/poolish_total, axis=1)
poolish['oz'] = poolish.apply(lambda row: float(row['grams']) / grams_per_ounce, axis=1)

poolish[['baker%', 'grams', 'oz']].style.format(formatter)

poolish weight = 621 grams


,baker%,grams,oz
flour,100,310,10.94
water,100,310,10.94
yeast,0.2,0.62,0.02


In [18]:
final_dough = formula.copy()
final_dough.at['poolish','grams'] = poolish_weight
final_dough.at['poolish','baker%'] = poolish_weight / flour_weight * 100
for index, row in poolish.iterrows():
    final_dough.at[index,'grams'] = final_dough.at[index,'grams'] - row['grams']
final_dough['oz'] = final_dough.apply(lambda row: row['grams']/grams_per_ounce, axis=1)
print("final dough")
final_dough[['baker%', 'grams', 'oz']].style.format(formatter)

final dough


,baker%,grams,oz
flour,100,630,22.22
water,66,310,10.94
yeast,0.4,2.79,0.10
salt,2.0,19,0.66
malt,0.5,4.70,0.17
poolish,66,621,21.91


In [27]:
display_df = final_dough.copy()
for col, fmt in formatter.items():
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(fmt)
print(display_df.to_markdown())

|         |   baker% |   grams |    oz |
|:--------|---------:|--------:|------:|
| flour   |    100   |  630    | 22.22 |
| water   |     66   |  310    | 10.94 |
| yeast   |      0.4 |    2.79 |  0.1  |
| salt    |      2   |   19    |  0.66 |
| malt    |      0.5 |    4.7  |  0.17 |
| poolish |     66   |  621    | 21.91 |
